In [8]:

import os
import cv2
import numpy as np
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.metrics import TopKCategoricalAccuracy
from tensorflow.keras import backend as K
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
def load_data(data_dir, img_size=(224, 224)):
    X = []
    y = []
    class_names = os.listdir(data_dir)
    for class_name in class_names:
        class_dir = os.path.join(data_dir, class_name)
        for img_name in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_name)
            img = cv2.imread(img_path)
            img = cv2.resize(img, img_size)
            X.append(img)
            y.append(class_names.index(class_name))
    return np.array(X), np.array(y), class_names

In [3]:

def create_model(num_classes):
    base_model = ResNet50(weights='imagenet', include_top=False)
    for layer in base_model.layers:
        layer.trainable = False
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dense(512, activation='relu')(x)
    output = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=base_model.input, outputs=output)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [4]:
def train_model(model, train_dir, val_dir, epochs=10, batch_size=32):
    train_datagen = ImageDataGenerator(
        preprocessing_function=preprocess_input,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True
    )
    
    val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
    
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='sparse'
    )
    
    val_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='sparse'
    )
    
    model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // batch_size,
        validation_data=val_generator,
        validation_steps=val_generator.samples // batch_size,
        epochs=epochs
    )
    
    return model

In [5]:
# Step 4: Face detection and recognition
def detect_and_recognize(image_path, model, class_names):
    face_cascade = cv2.CascadeClassifier(r"C:\Users\hp\Desktop\ml\haarcascade_frontalface_default.xml")
    img = cv2.imread(r"C:\Users\hp\Desktop\ml\what.jpeg")
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
    
    for (x, y, w, h) in faces:
        face = img[y:y+h, x:x+w]
        face = cv2.resize(face, (224, 224))
        face = np.expand_dims(face, axis=0)
        face = preprocess_input(face)
        
        prediction = model.predict(face)
        class_index = np.argmax(prediction)
        class_name = class_names[class_index]
        confidence = prediction[0][class_index]
        
        cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(img, f"{class_name} ({confidence:.2f})", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    
    cv2.imshow('Face Recognition', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [6]:
def evaluate_model(model, test_dir, batch_size=32):
    test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
    
    test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='sparse',
        shuffle=False
    )
    
    # Evaluate the model
    scores = model.evaluate(test_generator, verbose=1)
    print(f"Test loss: {scores[0]}")
    print(f"Test accuracy: {scores[1]*100:.2f}%")
    
    return scores[1] 

In [7]:
if __name__ == "__main__":
    train_dir = r'C:\Users\hp\Desktop\ml\dataset\train'
    test_dir = r'C:\Users\hp\Desktop\ml\dataset\val'
    
    # Load and preprocess data
    X_train, y_train, class_names = load_data(train_dir)
    X_test, y_test, _ = load_data(test_dir)
    
    # Create and compile the model
    model = create_model(len(class_names))
    
    # Train the model
    model = train_model(model, train_dir, test_dir)
    
    # Evaluate the model
    accuracy = evaluate_model(model, test_dir)
    
    # Save the model
    model.save('face_rec.h5')
    
    print(f"Model accuracy on test set: {accuracy*100:.2f}%")
    
    # Test the model on a sample image
    sample_image = 'Dataset/3.jpeg'
    detect_and_recognize(sample_image, model, class_names)

Found 93 images belonging to 5 classes.
Found 25 images belonging to 5 classes.
Epoch 1/10


C:\Users\hp\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


2/2 ━━━━━━━━━━━━━━━━━━━━ 11s 4s/step - accuracy: 0.3220 - loss: 2.1510 - val_accuracy: 0.4000 - val_loss: 3.0693
Epoch 2/10
1/2 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.4062 - loss: 3.1819

C:\Users\hp\AppData\Local\Programs\Python\Python310\lib\contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step - accuracy: 0.4062 - loss: 3.1819 - val_accuracy: 0.3600 - val_loss: 2.5748
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 2s/step - accuracy: 0.3854 - loss: 3.0414 - val_accuracy: 0.6400 - val_loss: 1.2205
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step - accuracy: 0.7931 - loss: 0.9958 - val_accuracy: 0.6400 - val_loss: 0.8679
Epoch 5/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 2s/step - accuracy: 0.7106 - loss: 0.7842 - val_accuracy: 0.6000 - val_loss: 1.0492
Epoch 6/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step - accuracy: 0.5938 - loss: 1.0484 - val_accuracy: 0.5200 - val_loss: 0.9869
Epoch 7/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 2s/step - accuracy: 0.7217 - loss: 0.7516 - val_accuracy: 0.8400 - val_loss: 0.4841
Epoch 8/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 981ms/step - accuracy: 0.9375 - loss: 0.3356 - val_accuracy: 0.8400 - val_loss: 0.4977
Epoch 9/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 2s/step - accuracy: 0.8600 - loss: 0.4260 - val_accuracy: 0.8000 - val_loss: 0.5035
Epoch 10/10
2/2 ━━━━━━━━

Test loss: 0.36965125799179077
Test accuracy: 88.00%
Model accuracy on test set: 88.00%
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
